### Module imports:

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from time import sleep
import glob
import numpy as np

## List of http links to jobs:

In [2]:
link_browser = ["https://www.linkedin.com/jobs/search/?f_WT=3&keywords=data%20scientist&location=%C4%8Desko", 
                "https://www.linkedin.com/jobs/search/?f_WT=3&keywords=data%20scientist&location=%C4%8Desko&start=25", 
                "https://www.linkedin.com/jobs/search/?f_WT=3&keywords=data%20scientist&location=%C4%8Desko&start=50", 
                "https://www.linkedin.com/jobs/search/?f_WT=3&keywords=data%20scientist&location=%C4%8Desko&start=75"]

In [3]:
size_of_list = len(link_browser)

links = []

for n in range(size_of_list):
    response = requests.get(link_browser[n])
    html_text = response.text
    soup = BeautifulSoup(html_text, "html.parser")
    hrefs = soup.find_all("a", class_= "base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")
    for n in hrefs:
        links.append(n["href"])

In [4]:
print(links)

['https://www.linkedin.com/jobs/view/data-scientist-risk-data-mining-at-tiktok-2955922873?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=24M1a1W6TqEW%2BwiPfSyK2A%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-at-now100-3076495367?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=IEqfN%2BW%2FLHydU2LrY4OuHw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-hybrid-urgent-at-sharp-decisions-3075722990?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=QI7mYt6F%2FddcJ0qEccD1RA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/sr-data-scientist-at-panera-bread-3067315160?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=0FNegrfc3qGU9acX9zw02Q%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-at-tatari-3030759116?refId=WT3BAb5f5nr6

## Data scraping from the list of links:

In [ ]:
size_of_list = len(links)
print("In process...")

d = {
    "name": [],
    "company_name": [],
    "ago": [],
    "contract": [],
    "location": [],
    "date": [],
    "description": [],
    "link": [],
    }

for n in range(size_of_list):
    print(n, links[n])
    response = requests.get(links[n])
    html_text = response.text
    
    #print(links[n])  
    
    soup = BeautifulSoup(html_text, "html.parser")
    try:
        name_s = soup.find("div", class_ ="top-card-layout__entity-info-container flex flex-wrap papabear:flex-nowrap")
        name_s = name_s.find("h1", class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
    except:
        name_s = soup.find("div", class_ ="top-card-layout__card relative p-2 papabear:p-details-container-padding")
        name_s = name_s.find("div", class_="top-card-layout__entity-info flex-grow flex-shrink-0 basis-0 babybear:flex-none babybear:w-full babybear:flex-none babybear:w-full")
        #name_s = name_s.find("h1", class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
        
    com_name_s = soup.find("span", class_ ="topcard__flavor")
    time_s = soup.find("span", class_="posted-time-ago__text topcard__flavor--metadata")
    description_s = soup.find("div", class_= "show-more-less-html__markup show-more-less-html__markup--clamp-after-5") 
    contract_s = soup.find("span", class_="description__job-criteria-text description__job-criteria-text--criteria")
    location_s = soup.find("span", class_="topcard__flavor topcard__flavor--bullet")
    try:
        salary_s = soup.find("div", class_="salary compensation__salary")
    except:
        pass
    
    try:
        name = name_s.text.strip().replace("++", "+")
    except:
        name = name_s.get_text.strip().replace("++", "+")
    company_name = com_name_s.text.strip()
    try:
        ago = time_s.get_text().strip()
    except:
        time_s = soup.find("span", class_="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata")
        ago = time_s.get_text().strip()
    description = description_s.text.strip()
    date = date.today()
    contract = contract_s.text.strip()
    location = location_s.text.strip()
    try:
        salary = salary_s.text.strip()
    except:
        pass
      
    d["name"].append(name)
    d["company_name"].append(company_name)
    d["ago"].append(ago)
    d["contract"].append(contract)
    d["description"].append(description)
    d["date"].append(date)
    d["link"].append(links[n])
    d["location"].append(location)
    try:
        print(salary)
    except:
        pass
    
    
    sleep(2)
    
    #print(name)
    #print(ago)
    #print(n, links[n])
print(f"Done: {n + 1} / {size_of_list}")

In process...
0 https://www.linkedin.com/jobs/view/data-scientist-risk-data-mining-at-tiktok-2955922873?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=24M1a1W6TqEW%2BwiPfSyK2A%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
$89,000.00/yr - $194,000.00/yr
1 https://www.linkedin.com/jobs/view/data-scientist-at-now100-3076495367?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=IEqfN%2BW%2FLHydU2LrY4OuHw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card
$89,000.00/yr - $194,000.00/yr
2 https://www.linkedin.com/jobs/view/data-scientist-hybrid-urgent-at-sharp-decisions-3075722990?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=QI7mYt6F%2FddcJ0qEccD1RA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card
$89,000.00/yr - $194,000.00/yr
3 https://www.linkedin.com/jobs/view/sr-data-scientist-at-panera-bread-3067315160?refId=WT3BAb5f5nr6x%2BVSqewX4g%3D%3D&trackingId=0FNegrfc3qGU9acX9zw02Q%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_

## Creation of CSV file out of web scraping: 

In [ ]:
data = pd.DataFrame.from_dict(data=d, orient='index')
data = data.transpose()

data.to_csv(f"RAW_DATA/scraped_data_{date}.csv", index=False)

## Creation of summary CSV file out of all CSV files

In [ ]:
# get data file names
path =r"C:\Users\srnam\PycharmProjects\data_analyz_pandas\LinkedIn_job_analysis\RAW_DATA"
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv("DATA/summary_data.csv", index=False)
big_frame.shape

In [ ]:
s = soup.find()
print(s)